In [4]:
from sklearn.feature_extraction.text import CountVectorizer
import lda
import numpy as np

In [13]:
def read_from_file(file_name):
    with open(file_name, 'r') as reader:
        line_list = reader.readlines()
    line_list = [x.strip() for x in line_list]
    return line_list

In [19]:
pros =read_from_file('project_pro.txt')
cons = read_from_file('project_con.txt')

Big company with a small company feel. Very entrepreneurial. Management facilitates your ideas that push the business forward


In [20]:
#pre-decided by the requirement of LDA algorithm
topic_num=5

#tokenization
pros_tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
cons_tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')                               
#read the dataset                
#docs=open('news.txt').readlines()

#transform the docs into a count matrix
#get teh vocabulary
pros_matrix = pros_tf_vectorizer.fit_transform(pros)
pros_vocab = pros_tf_vectorizer.get_feature_names()

cons_matrix = cons_tf_vectorizer.fit_transform(cons)
cons_vocab = cons_tf_vectorizer.get_feature_names()

#get the vocabulary
#vocab=tf_vectorizer.get_feature_names()

#initialize the LDA model
pro_model = lda.LDA(n_topics=topic_num, n_iter=500)
con_model = lda.LDA(n_topics=topic_num, n_iter=500)

#fit the model to the dataset
pro_model.fit(pros_matrix)
con_model.fit(cons_matrix)

INFO:lda:n_documents: 100
INFO:lda:vocab_size: 153
INFO:lda:n_words: 679
INFO:lda:n_topics: 5
INFO:lda:n_iter: 500
D:\miniconda\envs\tf\lib\site-packages\lda\utils.py:55: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if sparse and not np.issubdtype(doc_word.dtype, int):
INFO:lda:<0> log likelihood: -5953
INFO:lda:<10> log likelihood: -3976
INFO:lda:<20> log likelihood: -3932
INFO:lda:<30> log likelihood: -3882
INFO:lda:<40> log likelihood: -3878
INFO:lda:<50> log likelihood: -3843
INFO:lda:<60> log likelihood: -3833
INFO:lda:<70> log likelihood: -3853
INFO:lda:<80> log likelihood: -3867
INFO:lda:<90> log likelihood: -3842
INFO:lda:<100> log likelihood: -3881
INFO:lda:<110> log likelihood: -3837
INFO:lda:<120> log likelihood: -3828
INFO:lda:<130> log likelihood: -3835
INFO:lda:<140> log likelihood: -3825
INFO:lda:<150> log likelihood: -3830
INFO:lda:<160>

In [22]:
topic_keywords = {}
#write the top terms for each topic
#ancillary
top_words_num=3
pro_topic_mixes= pro_model.topic_word_
fw=open('pro_top_terms_per_topic.txt','w')
for i in range(topic_num):#for each topic
    top_indexes=np.argsort(pro_topic_mixes[i])[::-1][:top_words_num]                              
    my_top=''
    for ind in top_indexes:
        my_top+=pros_vocab[ind]+' '
        if i in topic_keywords:
            topic_keywords[i].append(pros_vocab[ind])
        else:
            topic_keywords[i] = [pros_vocab[ind]]
    fw.write('TOPIC: '+str(i)+' --> '+str(my_top)+'\n')
fw.close()

doc_topic = {}
doc_top_topic = {}
#write the top topics for each doc
top_topics_num=3
pro_doc_mixes= pro_model.doc_topic_
fw=open('pro_topic_mixture_per_doc.txt','w')
for i in range(len(pro_doc_mixes)):#for each doc
    top_indexes=np.argsort(pro_doc_mixes[i])[::-1][:top_topics_num]     
    my_top=''
    for ind in top_indexes:
        temp_topic = ind
        temp_likelihood = round(pro_doc_mixes[i][ind], 2)
        likelihood_top = -1
        topic_top = -1
        if temp_likelihood > likelihood_top:
            likelihood_top = temp_likelihood
            topic_top = temp_topic
        my_top+=' '+str(ind)+':'+str(round(pro_doc_mixes[i][ind],2))
        if i in doc_topic:
            doc_topic[i].append((ind, round(pro_doc_mixes[i][ind], 2)))
        else:
            doc_topic[i] = [(ind, round(pro_doc_mixes[i][ind], 2))]
    doc_top_topic[i] = topic_top
    fw.write('DOC: '+str(i)+' --> '+str(my_top)+'\n')
fw.close()

[148  59  58]
-------------------- 153
148
59
58
[104  76  98]
-------------------- 153
104
76
98
[ 29  22 150]
-------------------- 153
29
22
150
[  5 105 113]
-------------------- 153
5
105
113
[ 58  81 104]
-------------------- 153
58
81
104


In [23]:
print(topic_keywords)
print(doc_top_topic)

k_user = 2 #user parameter
invers_count = {}
for key, value in doc_top_topic.items():
    if value in invers_count:
        invers_count[value] += 1
    else:
        invers_count[value] = 1
print(invers_count)
count_dic = {}
for k,v in invers_count.items():
    count_dic[v] = k
print(count_dic)

for i in range(k_user):    
    l = len(count_dic.keys())
    i_topicNum = count_dic[sorted(count_dic.keys())[l-i-1]]                
    print("TOP ", i+1, " pro comments' keywords are ", topic_keywords[i_topicNum])
    
    for docj, portion in doc_topic.items():
        for j, posb in portion:
            if j == i_topicNum and posb > 0.8:
                print("e.g. ", pros[docj])

{0: ['work', 'great', 'good'], 1: ['people', 'job', 'opportunities'], 2: ['day', 'company', 'working'], 3: ['amazon', 'place', 'really'], 4: ['good', 'learn', 'people']}
{0: 0, 1: 3, 2: 3, 3: 3, 4: 3, 5: 1, 6: 0, 7: 3, 8: 1, 9: 2, 10: 0, 11: 4, 12: 3, 13: 3, 14: 4, 15: 2, 16: 2, 17: 3, 18: 4, 19: 3, 20: 4, 21: 0, 22: 4, 23: 4, 24: 3, 25: 4, 26: 3, 27: 3, 28: 2, 29: 3, 30: 2, 31: 2, 32: 4, 33: 3, 34: 3, 35: 4, 36: 3, 37: 3, 38: 2, 39: 3, 40: 3, 41: 3, 42: 3, 43: 3, 44: 3, 45: 3, 46: 2, 47: 3, 48: 4, 49: 0, 50: 2, 51: 3, 52: 3, 53: 3, 54: 3, 55: 4, 56: 2, 57: 3, 58: 3, 59: 3, 60: 3, 61: 4, 62: 1, 63: 2, 64: 3, 65: 3, 66: 3, 67: 3, 68: 4, 69: 2, 70: 3, 71: 4, 72: 4, 73: 4, 74: 3, 75: 0, 76: 2, 77: 4, 78: 1, 79: 2, 80: 3, 81: 0, 82: 3, 83: 2, 84: 2, 85: 3, 86: 3, 87: 4, 88: 3, 89: 4, 90: 3, 91: 3, 92: 4, 93: 4, 94: 3, 95: 3, 96: 3, 97: 4, 98: 1, 99: 3}
{0: 7, 3: 50, 1: 5, 2: 16, 4: 22}
{7: 0, 50: 3, 5: 1, 16: 2, 22: 4}
TOP  1 comments' keywords are  ['people', 'job', 'opportunities']
TOP  

In [30]:
topic_keywords = {}
#write the top terms for each topic
#ancillary
top_words_num=3
con_topic_mixes= con_model.topic_word_
fw=open('con_top_terms_per_topic.txt','w')
for i in range(topic_num):#for each topic
    top_indexes=np.argsort(con_topic_mixes[i])[::-1][:top_words_num]                              
    my_top=''
    for ind in top_indexes:
        my_top+=cons_vocab[ind]+' '
        if i in topic_keywords:
            topic_keywords[i].append(cons_vocab[ind])
        else:
            topic_keywords[i] = [cons_vocab[ind]]
    fw.write('TOPIC: '+str(i)+' --> '+str(my_top)+'\n')
fw.close()

doc_topic = {}
doc_top_topic = {}
#write the top topics for each doc
top_topics_num=3
con_doc_mixes= con_model.doc_topic_
fw=open('con_topic_mixture_per_doc.txt','w')
for i in range(len(con_doc_mixes)):#for each doc
    top_indexes=np.argsort(con_doc_mixes[i])[::-1][:top_topics_num]     
    my_top=''
    for ind in top_indexes:
        temp_topic = ind
        temp_likelihood = round(con_doc_mixes[i][ind], 2)
        likelihood_top = -1
        topic_top = -1
        if temp_likelihood > likelihood_top:
            likelihood_top = temp_likelihood
            topic_top = temp_topic
        my_top+=' '+str(ind)+':'+str(round(con_doc_mixes[i][ind],2))
        if i in doc_topic:
            doc_topic[i].append((ind, round(con_doc_mixes[i][ind], 2)))
        else:
            doc_topic[i] = [(ind, round(con_doc_mixes[i][ind], 2))]
    doc_top_topic[i] = topic_top
    fw.write('DOC: '+str(i)+' --> '+str(my_top)+'\n')
fw.close()

print(topic_keywords)
print(doc_top_topic)

k_user = 2 #user parameter
invers_count = {}
for key, value in doc_top_topic.items():
    if value in invers_count:
        invers_count[value] += 1
    else:
        invers_count[value] = 1
print(invers_count)
count_dic = {}
for k,v in invers_count.items():
    count_dic[v] = k
print(count_dic)

for i in range(k_user):
    l = len(count_dic.keys())
    i_topicNum = count_dic[sorted(count_dic.keys())[l-i-1]]                
    print("TOP ", i+1, " con comments' keywords are ", topic_keywords[i_topicNum])
    
    for docj, portion in doc_topic.items():
        for j, posb in portion:
            if j == i_topicNum and posb > 0.8:
                print("e.g. ", cons[docj])

{0: ['fast', 'environment', 'hard'], 1: ['email', 'continue', 'reading'], 2: ['work', 'time', 'long'], 3: ['company', 'bad', 'management'], 4: ['email', 'resend', 'continue']}
{0: 4, 1: 3, 2: 2, 3: 4, 4: 2, 5: 2, 6: 4, 7: 3, 8: 0, 9: 2, 10: 4, 11: 2, 12: 3, 13: 2, 14: 0, 15: 3, 16: 3, 17: 2, 18: 2, 19: 2, 20: 2, 21: 3, 22: 1, 23: 4, 24: 4, 25: 4, 26: 4, 27: 3, 28: 2, 29: 4, 30: 4, 31: 3, 32: 3, 33: 3, 34: 2, 35: 3, 36: 3, 37: 4, 38: 3, 39: 2, 40: 2, 41: 4, 42: 4, 43: 3, 44: 4, 45: 3, 46: 3, 47: 3, 48: 2, 49: 0, 50: 3, 51: 3, 52: 3, 53: 3, 54: 4, 55: 2, 56: 3, 57: 2, 58: 2, 59: 3, 60: 2, 61: 2, 62: 2, 63: 2, 64: 3, 65: 4, 66: 2, 67: 2, 68: 3, 69: 3, 70: 3, 71: 3, 72: 2, 73: 4, 74: 3, 75: 1, 76: 3, 77: 2, 78: 0, 79: 2, 80: 3, 81: 4, 82: 3, 83: 4, 84: 1, 85: 2, 86: 4, 87: 2, 88: 3, 89: 2, 90: 4, 91: 3, 92: 4, 93: 4, 94: 2, 95: 3, 96: 2, 97: 2, 98: 4, 99: 2}
{4: 24, 3: 35, 2: 34, 0: 4, 1: 3}
{24: 4, 35: 3, 34: 2, 4: 0, 3: 1}
TOP  1  con comments' keywords are  ['company', 'bad', 'managemen